In [1]:
from tpot import  TPOTClassifier
from sklearn.model_selection import train_test_split
from __future__ import print_function
import sys,tempfile, urllib, os
import pandas as pd
import numpy as np

e:\ML PROGRAMS\AUTOML-TPOT\myenv\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [4]:
data = pd.read_csv("diamonds.csv")

In [5]:
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [6]:
# Data Processing:
#Dropping dimentionless diamonds
data = data.drop(data[data["x"]==0].index)
data = data.drop(data[data["y"]==0].index)
data = data.drop(data[data["z"]==0].index)

In [7]:
# Remove all the outliers in the datset:
#Dropping the outliers. 
data = data[(data["depth"]<75)&(data["depth"]>45)]
data = data[(data["table"]<80)&(data["table"]>40)]
data = data[(data["x"]<30)]
data = data[(data["y"]<30)]
data = data[(data["z"]<30)&(data["z"]>2)]

In [8]:
s = (data.dtypes =="object")
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

Categorical variables:
['cut', 'color', 'clarity']


In [9]:
# Apply Encoding
from sklearn.preprocessing import LabelEncoder
label_data = data.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    label_data[col] = label_encoder.fit_transform(label_data[col])

In [10]:
X= label_data.drop(["price"],axis =1)
y= label_data["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=7)

In [11]:
X_train

,carat,cut,color,clarity,depth,table,x,y,z
8778,0.90,1,0,2,58.5,63.0,6.21,6.30,3.66
47001,0.53,2,3,7,62.5,54.0,5.19,5.21,3.25
40421,0.50,0,4,4,66.6,58.0,4.95,4.84,3.26
44023,0.51,2,1,2,62.2,57.0,5.12,5.08,3.17
13402,0.32,3,4,7,62.2,58.0,4.33,4.38,2.71
...,...,...,...,...,...,...,...,...,...
919,0.72,0,2,4,56.9,69.0,5.93,5.77,3.33
53492,0.75,3,2,2,61.5,58.0,5.83,5.87,3.60
38492,0.42,3,2,4,61.2,58.0,4.82,4.86,2.96
10750,1.12,4,3,3,60.2,59.0,6.70,6.85,4.08


In [12]:
tpot =  TPOTClassifier(generations=4, population_size=10, verbosity=3)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

32 operators have been imported by TPOT.
Skipped pipeline #1 due to time out. Continuing to the next pipeline.         
Skipped pipeline #5 due to time out. Continuing to the next pipeline.         
Skipped pipeline #7 due to time out. Continuing to the next pipeline.         
Skipped pipeline #10 due to time out. Continuing to the next pipeline.         
Skipped pipeline #12 due to time out. Continuing to the next pipeline.         
Skipped pipeline #15 due to time out. Continuing to the next pipeline.         
_pre_test decorator: _random_mutation_operator: num_test=0 Invalid classes inferred from unique values of `y`.  Expected: [    0     1     2 ... 10404 10405 10406], got [  326   334   335 ... 18806 18818 18823].
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Skipped pipeline #18 due to time out. Continuing to the next pipeline.         
Skipped pipeline #20 due to time out. Continuing to the

In [13]:
print(tpot.score(X_test, y_test))

0.08503376122282406


In [14]:
tpot.export('tpot_diamond_pipeline.py')

In [15]:
tpot.evaluated_individuals_


{'MLPClassifier(Normalizer(input_matrix, Normalizer__norm=l1), MLPClassifier__alpha=0.001, MLPClassifier__learning_rate_init=0.5)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 2,
  'internal_cv_score': -inf},
 'MLPClassifier(SelectFromModel(input_matrix, SelectFromModel__ExtraTreesClassifier__criterion=entropy, SelectFromModel__ExtraTreesClassifier__max_features=1.0, SelectFromModel__ExtraTreesClassifier__n_estimators=100, SelectFromModel__threshold=0.7000000000000001), MLPClassifier__alpha=0.0001, MLPClassifier__learning_rate_init=0.01)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  'operator_count': 2,
  'internal_cv_score': -inf},
 'KNeighborsClassifier(input_matrix, KNeighborsClassifier__n_neighbors=60, KNeighborsClassifier__p=2, KNeighborsClassifier__weights=distance)': {'generation': 0,
  'mutation_count': 0,
  'crossover_count': 0,
  'predecessor': ('ROOT',),
  

In [18]:
tpot.fitted_pipeline_

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('kneighborsclassifier',
                 KNeighborsClassifier(n_neighbors=2, weights='distance'))])